In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_chroma import Chroma

In [2]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "feats_with_books_and_categories.csv"

# Load the latest version
feats_df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "tedcoderman/pathfinder-1e-feats-and-categories",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

feats_df

/media/deltreey/space/git/pathfinder1e-llm/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_3271902/1479633376.py:10: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  feats_df = kagglehub.load_dataset(


,URI,Name,Prerequisites,Short Description,long_description,Books,Category
0,../advancedClassGuide/feats.html#aberrant-tumor,Aberrant Tumor,Aberrant bloodline,Gain a tumor familiar,"To the surprise of others, that strange growth...",Advanced Class Guide,Untyped
1,../bestiary/monsterFeats.html#ability-focus,Ability Focus,Special attack,Add +2 to the DC for all saving throws against...,One of this creature's special attacks is part...,Bestiary,Untyped
2,../ultimateMagic/ultimateMagicFeats.html#abund...,Abundant Revelations,Mystery class feature,Use a single revelation more times each day,You can plumb the depths of your mystery to us...,Ultimate Magic,Untyped
3,../ultimateCampaign/characterBackground/storyF...,Accursed,Carry a curse or have the Cursed Birth background,Gain SR 5 + character level,Your curse weighs down your soul like a millst...,Ultimate Campaign,Story
4,../ultimateMagic/ultimateMagicFeats.html#accur...,Accursed Hex,Hex class feature,Target a creature with a hex a second time tha...,You can make a second attempt at failed hexes....,Ultimate Magic,Untyped
...,...,...,...,...,...,...,...
1401,../ultimateMagic/ultimateMagicFeats.html#witch...,Witch Knife,Witch class,Use dagger as focus for witch patron spells,You empower your witch spells by incorporating...,Ultimate Magic,Untyped
1402,../mythicAdventures/mythicFeats.html#witch-kni...,Witch Knife - Mythic,Witch Knife,Use your witch knife as a focus for all spells...,Your witch knife is as much an extension of yo...,Mythic Adventures,Mythic
1403,../ultimateMagic/ultimateMagicFeats.html#word-...,Word of Healing,Lay on hands class feature,Use lay on hands from a distance,Using the same divine energy as your lay on ha...,Ultimate Magic,Untyped
1404,../technologyGuide/feats.html#wrest-charge,Wrest Charge,"Disable Device 5 ranks, Knowledge (engineering...",Jury-rig a depleted battery to hold 1 charge,You can extract a charge from otherwise deplet...,Technology Guide,Untyped


In [3]:
import numpy as np

feats_df['has_benefit'] = (np.where(feats_df['long_description'].str.contains('Benefit:'), 1, np.where(feats_df['long_description'].str.contains('Benefits:'), 1, 0)))
feats_df['has_benefit'].sum()

1398

In [4]:
feats_df[feats_df['has_benefit'] == 0]

,URI,Name,Prerequisites,Short Description,long_description,Books,Category,has_benefit
211,../advancedRaceGuide/uncommonRaces/merfolk.htm...,Sea Hunter,"Combat Expertise, merfolk",Attempt to knock a target off balance with a s...,Your blows knock swimming opponents off balance.,Advanced Race Guide,Combat,0
285,../ultimateCombat/ultimateCombatFeats.html#dea...,Deadly Finish,Base attack bonus +11,Foes you knock out must make a Fort save or die,Your attacks don't just fell your opponents—th...,Ultimate Combat,Combat,0
306,../advancedRaceGuide/featuredRaces/hobgoblins....,Terrorizing Display,"Dazzling Display, Demoralizing Lash, Taskmaste...","When you use Dazzling Display, you can use it ...",Your battle prowess frightens friend and foe a...,Advanced Race Guide,Combat,0
553,../advancedRaceGuide/uncommonRaces/duergar.htm...,Giant Steps,"Duergar, slow and steady racial trait",Increase your base speed by 20 feet when your ...,"When enlarged, your speed increases.",Advanced Race Guide,Untyped,0
752,../advancedRaceGuide/uncommonRaces/sulis.html#...,Incremental Elemental Assault,Suli,Use elemental assault in 1-round increments,You may activate and quench your elemental ass...,Advanced Race Guide,Untyped,0
1120,../advancedPlayersGuide/advancedFeats.html#sav...,Saving Shield,Shield Proficiency,Grant shield bonus to an adjacent ally,You deflect attacks that could mean your ally'...,Advanced Player's Guide,Combat,0
1158,../advancedRaceGuide/uncommonRaces/strix.html#...,Stretched Wings,"Str 13, Skill Focus (Fly), strix, wing-clipped...",Strix racial fly speed increases to 60 feet,You strengthen your crippled wings.,Advanced Race Guide,Untyped,0
1289,../advancedRaceGuide/uncommonRaces/vanaras.htm...,Tree Hanger,"Acrobatics 1 rank, vanara",Gain a +2 bonus on your CMD against all trip a...,You can use your tail to defend against trip a...,Advanced Race Guide,Combat,0


In [13]:
from tqdm import tqdm

def get_benefit(description: str):
    if type(description) != str:
        return None
    if 'Benefit:' in description:
        return description.split('Benefit:')[1].strip()
    elif 'Benefits:' in description:
        return description.split('Benefits:')[1].strip()
    else:
        return None

benefits = []
for i in tqdm(range(len(feats_df))):
    benefits.append(get_benefit(feats_df['long_description'][i]))
benefits

100%|██████████| 1406/1406 [00:00<00:00, 129030.09it/s]


["You gain a tumor familiar, as the tumor familiar alchemist discovery, with an effective alchemist level equal to the level of the class that grants your aberrant bloodline for determining the tumor familiar's abilities. If multiple classes grant you the aberrant bloodline, those class levels stack for determining your effective alchemist level.",
 "Choose one of the creature's special attacks. Add +2 to the DC for all saving throws against the special attack on which the creature focuses.\n\nSpecial: A creature can gain this feat multiple times. Its effects do not stack. Each time the creature takes the feat, it applies to a different special attack.",
 'Choose one of your revelations that has a number of uses per day. You gain 1 additional use per day of that revelation.\n\nSpecial: You can gain this feat multiple times. Its effects do not stack. Each time you take the feat, it applies to a new revelation.',
 "You gain spell resistance equal to 5 + your character level, as the curse

In [14]:
feats_df['benefit'] = benefits
feats_df

,URI,Name,Prerequisites,Short Description,long_description,Books,Category,has_benefit,benefit
0,../advancedClassGuide/feats.html#aberrant-tumor,Aberrant Tumor,Aberrant bloodline,Gain a tumor familiar,"To the surprise of others, that strange growth...",Advanced Class Guide,Untyped,1,"You gain a tumor familiar, as the tumor famili..."
1,../bestiary/monsterFeats.html#ability-focus,Ability Focus,Special attack,Add +2 to the DC for all saving throws against...,One of this creature's special attacks is part...,Bestiary,Untyped,1,Choose one of the creature's special attacks. ...
2,../ultimateMagic/ultimateMagicFeats.html#abund...,Abundant Revelations,Mystery class feature,Use a single revelation more times each day,You can plumb the depths of your mystery to us...,Ultimate Magic,Untyped,1,Choose one of your revelations that has a numb...
3,../ultimateCampaign/characterBackground/storyF...,Accursed,Carry a curse or have the Cursed Birth background,Gain SR 5 + character level,Your curse weighs down your soul like a millst...,Ultimate Campaign,Story,1,You gain spell resistance equal to 5 + your ch...
4,../ultimateMagic/ultimateMagicFeats.html#accur...,Accursed Hex,Hex class feature,Target a creature with a hex a second time tha...,You can make a second attempt at failed hexes....,Ultimate Magic,Untyped,1,When you target a creature with a hex that can...
...,...,...,...,...,...,...,...,...,...
1401,../ultimateMagic/ultimateMagicFeats.html#witch...,Witch Knife,Witch class,Use dagger as focus for witch patron spells,You empower your witch spells by incorporating...,Ultimate Magic,Untyped,1,"Each day, when you prepare your spells, you ca..."
1402,../mythicAdventures/mythicFeats.html#witch-kni...,Witch Knife - Mythic,Witch Knife,Use your witch knife as a focus for all spells...,Your witch knife is as much an extension of yo...,Mythic Adventures,Mythic,1,You can use your witch knife as an additional ...
1403,../ultimateMagic/ultimateMagicFeats.html#word-...,Word of Healing,Lay on hands class feature,Use lay on hands from a distance,Using the same divine energy as your lay on ha...,Ultimate Magic,Untyped,1,You may use your lay on hands to heal another ...
1404,../technologyGuide/feats.html#wrest-charge,Wrest Charge,"Disable Device 5 ranks, Knowledge (engineering...",Jury-rig a depleted battery to hold 1 charge,You can extract a charge from otherwise deplet...,Technology Guide,Untyped,1,"With a successful DC 20 Disable Device check, ..."


In [15]:
feats_df['benefit'] = (np.where(feats_df['benefit'].isna(), feats_df['Short Description'], feats_df['benefit']))
feats_df

,URI,Name,Prerequisites,Short Description,long_description,Books,Category,has_benefit,benefit
0,../advancedClassGuide/feats.html#aberrant-tumor,Aberrant Tumor,Aberrant bloodline,Gain a tumor familiar,"To the surprise of others, that strange growth...",Advanced Class Guide,Untyped,1,"You gain a tumor familiar, as the tumor famili..."
1,../bestiary/monsterFeats.html#ability-focus,Ability Focus,Special attack,Add +2 to the DC for all saving throws against...,One of this creature's special attacks is part...,Bestiary,Untyped,1,Choose one of the creature's special attacks. ...
2,../ultimateMagic/ultimateMagicFeats.html#abund...,Abundant Revelations,Mystery class feature,Use a single revelation more times each day,You can plumb the depths of your mystery to us...,Ultimate Magic,Untyped,1,Choose one of your revelations that has a numb...
3,../ultimateCampaign/characterBackground/storyF...,Accursed,Carry a curse or have the Cursed Birth background,Gain SR 5 + character level,Your curse weighs down your soul like a millst...,Ultimate Campaign,Story,1,You gain spell resistance equal to 5 + your ch...
4,../ultimateMagic/ultimateMagicFeats.html#accur...,Accursed Hex,Hex class feature,Target a creature with a hex a second time tha...,You can make a second attempt at failed hexes....,Ultimate Magic,Untyped,1,When you target a creature with a hex that can...
...,...,...,...,...,...,...,...,...,...
1401,../ultimateMagic/ultimateMagicFeats.html#witch...,Witch Knife,Witch class,Use dagger as focus for witch patron spells,You empower your witch spells by incorporating...,Ultimate Magic,Untyped,1,"Each day, when you prepare your spells, you ca..."
1402,../mythicAdventures/mythicFeats.html#witch-kni...,Witch Knife - Mythic,Witch Knife,Use your witch knife as a focus for all spells...,Your witch knife is as much an extension of yo...,Mythic Adventures,Mythic,1,You can use your witch knife as an additional ...
1403,../ultimateMagic/ultimateMagicFeats.html#word-...,Word of Healing,Lay on hands class feature,Use lay on hands from a distance,Using the same divine energy as your lay on ha...,Ultimate Magic,Untyped,1,You may use your lay on hands to heal another ...
1404,../technologyGuide/feats.html#wrest-charge,Wrest Charge,"Disable Device 5 ranks, Knowledge (engineering...",Jury-rig a depleted battery to hold 1 charge,You can extract a charge from otherwise deplet...,Technology Guide,Untyped,1,"With a successful DC 20 Disable Device check, ..."


In [16]:
feats_df['tagged_benefit'] = feats_df.index.astype(str)+ '|' + feats_df['benefit'].astype(str)
feats_df['tagged_benefit'] = feats_df['tagged_benefit'].apply(lambda t: t.replace("\n", " "))
feats_df

,URI,Name,Prerequisites,Short Description,long_description,Books,Category,has_benefit,benefit,tagged_benefit
0,../advancedClassGuide/feats.html#aberrant-tumor,Aberrant Tumor,Aberrant bloodline,Gain a tumor familiar,"To the surprise of others, that strange growth...",Advanced Class Guide,Untyped,1,"You gain a tumor familiar, as the tumor famili...","0|You gain a tumor familiar, as the tumor fami..."
1,../bestiary/monsterFeats.html#ability-focus,Ability Focus,Special attack,Add +2 to the DC for all saving throws against...,One of this creature's special attacks is part...,Bestiary,Untyped,1,Choose one of the creature's special attacks. ...,1|Choose one of the creature's special attacks...
2,../ultimateMagic/ultimateMagicFeats.html#abund...,Abundant Revelations,Mystery class feature,Use a single revelation more times each day,You can plumb the depths of your mystery to us...,Ultimate Magic,Untyped,1,Choose one of your revelations that has a numb...,2|Choose one of your revelations that has a nu...
3,../ultimateCampaign/characterBackground/storyF...,Accursed,Carry a curse or have the Cursed Birth background,Gain SR 5 + character level,Your curse weighs down your soul like a millst...,Ultimate Campaign,Story,1,You gain spell resistance equal to 5 + your ch...,3|You gain spell resistance equal to 5 + your ...
4,../ultimateMagic/ultimateMagicFeats.html#accur...,Accursed Hex,Hex class feature,Target a creature with a hex a second time tha...,You can make a second attempt at failed hexes....,Ultimate Magic,Untyped,1,When you target a creature with a hex that can...,4|When you target a creature with a hex that c...
...,...,...,...,...,...,...,...,...,...,...
1401,../ultimateMagic/ultimateMagicFeats.html#witch...,Witch Knife,Witch class,Use dagger as focus for witch patron spells,You empower your witch spells by incorporating...,Ultimate Magic,Untyped,1,"Each day, when you prepare your spells, you ca...","1401|Each day, when you prepare your spells, y..."
1402,../mythicAdventures/mythicFeats.html#witch-kni...,Witch Knife - Mythic,Witch Knife,Use your witch knife as a focus for all spells...,Your witch knife is as much an extension of yo...,Mythic Adventures,Mythic,1,You can use your witch knife as an additional ...,1402|You can use your witch knife as an additi...
1403,../ultimateMagic/ultimateMagicFeats.html#word-...,Word of Healing,Lay on hands class feature,Use lay on hands from a distance,Using the same divine energy as your lay on ha...,Ultimate Magic,Untyped,1,You may use your lay on hands to heal another ...,1403|You may use your lay on hands to heal ano...
1404,../technologyGuide/feats.html#wrest-charge,Wrest Charge,"Disable Device 5 ranks, Knowledge (engineering...",Jury-rig a depleted battery to hold 1 charge,You can extract a charge from otherwise deplet...,Technology Guide,Untyped,1,"With a successful DC 20 Disable Device check, ...",1404|With a successful DC 20 Disable Device ch...


In [17]:
feats_df['tagged_benefit'][0]

"0|You gain a tumor familiar, as the tumor familiar alchemist discovery, with an effective alchemist level equal to the level of the class that grants your aberrant bloodline for determining the tumor familiar's abilities. If multiple classes grant you the aberrant bloodline, those class levels stack for determining your effective alchemist level."

In [18]:
feats_df["tagged_benefit"].to_csv("tagged_benefit.txt", sep = "\n", index = False, header = False)

In [19]:
raw_documents = TextLoader("tagged_benefit.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 348, which is longer than the specified 0
Created a chunk of size 307, which is longer than the specified 0
Created a chunk of size 257, which is longer than the specified 0
Created a chunk of size 437, which is longer than the specified 0
Created a chunk of size 453, which is longer than the specified 0
Created a chunk of size 160, which is longer than the specified 0
Created a chunk of size 155, which is longer than the specified 0
Created a chunk of size 260, which is longer than the specified 0
Created a chunk of size 282, which is longer than the specified 0
Created a chunk of size 321, which is longer than the specified 0
Created a chunk of size 276, which is longer than the specified 0
Created a chunk of size 284, which is longer than the specified 0
Created a chunk of size 205, which is longer than the specified 0
Created a chunk of size 231, which is longer than the specified 0
Created a chunk of size 215, which is longer than the specified 0
Created a 

In [20]:
documents[0]

Document(metadata={'source': 'tagged_benefit.txt'}, page_content="0|You gain a tumor familiar, as the tumor familiar alchemist discovery, with an effective alchemist level equal to the level of the class that grants your aberrant bloodline for determining the tumor familiar's abilities. If multiple classes grant you the aberrant bloodline, those class levels stack for determining your effective alchemist level.")

In [21]:
db_feats = Chroma.from_documents(documents, embedding=GPT4AllEmbeddings())

Downloading: 100%|██████████| 45.9M/45.9M [00:50<00:00, 912kiB/s] 
Verifying: 100%|██████████| 45.9M/45.9M [00:00<00:00, 524MiB/s]
Failed to load libllamamodel-mainline-cuda.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory
Failed to load libllamamodel-mainline-cuda-avxonly.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [23]:
query = "A feat  to give me a bonus to Acrobatics"
docs = db_feats.similarity_search(query, k=10)
docs

[Document(id='4d0527fd-763b-4562-8923-20a593951ac5', metadata={'source': 'tagged_benefit.txt'}, page_content='787|Once per day as a swift action, you can conjure a burst of energy from your kinetic blast to help you jump a long distance, adding a +10 bonus on your Acrobatics check to jump; if you have at least 10 ranks in Acrobatics, the bonus increases to +20. By accepting 1 point of burn, you can use this ability at will until your burn is removed.'),
 Document(id='a5bd6b47-9dac-4a9e-a0ad-67827a67a1ee', metadata={'source': 'tagged_benefit.txt'}, page_content="349|When you use Acrobatics to move through an opponent's threatened area or space without provoking an attack of opportunity from that opponent, you gain a +2 circumstance bonus on your next melee attack roll against that opponent and that opponent is denied its Dexterity bonus to AC, as long as you make that attack before the start of your next turn."),
 Document(id='6e00f530-8402-46d8-9945-1207b46bac9f', metadata={'source': '

In [24]:
feats_df.iloc[int(docs[0].page_content.split('|')[0].strip())]

URI                        ../occultAdventures/feats.html#kinetic-leap
Name                                                      Kinetic Leap
Prerequisites          Acrobatics 3 ranks, kinetic blast class feature
Short Description    Use blast to add +10 on Acrobatics checks to j...
long_description     Kinetic energy propels you when you jump.\n\nP...
Books                                                Occult Adventures
Category                                                       Untyped
has_benefit                                                          1
benefit              Once per day as a swift action, you can conjur...
tagged_benefit       787|Once per day as a swift action, you can co...
Name: 787, dtype: object

In [26]:
import pandas as pd

def recommmend_feats(query: str, top_k: int = 10) -> pd.DataFrame:
    recs = db_feats.similarity_search(query, k=top_k)
    feat_list = []
    for i in range(len(recs)):
        feat_list.append(int(recs[i].page_content.split('|')[0].strip()))
    return feats_df.filter(items=feat_list, axis=0)

In [27]:
recommmend_feats('a feat that gives extra sneak attack damage')

,URI,Name,Prerequisites,Short Description,long_description,Books,Category,has_benefit,benefit,tagged_benefit
271,../ultimateCombat/ultimateCombatFeats.html#sne...,Sneaking Precision,"Sneak attack class feature, Critical Focus, an...","On the second sneak attack in a round, you may...",Your knowledge of your enemies' vulnerable spo...,Ultimate Combat,Untyped,1,Whenever you successfully sneak attack an oppo...,271|Whenever you successfully sneak attack an ...
1071,../monsterCodex/ogres.html#savage-critical,Savage Critical,"Str 19, ogre, sneak attack +2d6",Add sneak attack damage to confirmed critical ...,"You are able to land critical hits precisely, ...",Monster Codex,Untyped,1,When you hit a creature using the Vital Strike...,1071|When you hit a creature using the Vital S...
995,../advancedPlayersGuide/advancedFeats.html#pre...,Precise Strike,"Dex 13, base attack bonus +1",Deal +1d6 points of precision damage with mele...,"You are skilled at striking where it counts, a...",Advanced Player's Guide,"Combat,Teamwork",1,Whenever you and an ally who also has this fea...,995|Whenever you and an ally who also has this...
1082,../advancedClassGuide/feats.html#extreme-preju...,Extreme Prejudice,"Seething Hatred, sneak attack +3d6, studied ta...",When you succeed at a sneak attack with Seethi...,Your hatred of a particular type of creature h...,Advanced Class Guide,Combat,1,When you succeed at a sneak attack against a c...,1082|When you succeed at a sneak attack agains...
1069,../ultimateCombat/ultimateCombatFeats.html#sap...,Sap Adept,Sneak attack +1d6,Gain bonus damage on nonlethal attacks,You know just where to hit to knock the sense ...,Ultimate Combat,Combat,1,Whenever you use a bludgeoning weapon to deal ...,1069|Whenever you use a bludgeoning weapon to ...
1094,../advancedPlayersGuide/advancedFeats.html#sha...,Shadow Strike,Base attack bonus +1,Deal precision damage against targets with con...,You accurately strike even those you cannot cl...,Advanced Player's Guide,Combat,1,"You can deal precision damage, such as sneak a...","1094|You can deal precision damage, such as sn..."
1070,../ultimateCombat/ultimateCombatFeats.html#sap...,Sap Master,"Sap Adept, sneak attack +3d6",Deal twice your sneak attack damage when deali...,You knock the sense out of foes with a well-ti...,Ultimate Combat,Combat,1,Whenever you use a bludgeoning weapon to deal ...,1070|Whenever you use a bludgeoning weapon to ...
219,../coreRulebook/feats.html#combat-reflexes,Combat Reflexes,—,Make additional attacks of opportunity,You can make additional attacks of opportunity...,Core Rulebook,Combat,1,You may make a number of additional attacks of...,219|You may make a number of additional attack...
643,http://paizo.com/pathfinder/prd/ultimateCombat...,Crane Wing,"Crane Style, base attack bonus +5 or monk level 5",May deflect one attack per round while fightin...,You move with the speed and finesse of an avia...,Ultimate Combat,"Combat,Style",1,When fighting defensively with at least one ha...,643|When fighting defensively with at least on...
701,../ultimateCombat/ultimateCombatFeats.html#kno...,Knockout Artist,"Sneak attack class feature, Improved Unarmed S...",Gain +2 damage per sneak attack when dealing n...,You can throw devastating knockout punches.\n\...,Ultimate Combat,Untyped,1,When you use your unarmed strike to deal nonle...,701|When you use your unarmed strike to deal n...


In [28]:
feats_df.drop(['URI', 'has_benefit', 'tagged_benefit'], axis=1).to_csv("feats_clean.csv", index = True)